# Deep Q Network

## 1. Setup

In [117]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v0').unwrapped

state = env.reset()
print(state)

action = env.action_space.sample()
print(action)

n_action = env.action_space.n
n_state = env.observation_space.shape[0]
print(n_state)
print(n_action)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion() # interactive mode on: 交互模式

[-0.0279861  -0.01223715  0.02821148  0.0349384 ]
1
4
2


## 2. Replay Memory

In [124]:
Experience = namedtuple('Experience', ('state', 'action', 'reward', 'next_state', 'terminal'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = -1

    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None) # allocate space
        self.position = (self.position + 1) % self.capacity
        self.memory[self.position] = Experience(*args) ## append a new experience

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self): ## len(experience)
        return len(self.memory)


experience_pool = ReplayMemory(10000) #initialize memory pool

## 3. Q-Network

In [97]:
class NN():
    def __init__(self, input_size, output_size):
        super(NN, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(input_size, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, output_size))

policy_net = NN(n_state,n_action).model #initialize nn models
target_net = NN(n_state,n_action).model
target_net.load_state_dict(policy_net.state_dict()) ## copy policy to target

<All keys matched successfully>

## 4. Hyperparameters


In [28]:
BATCH_SIZE = 128 #batch-train
GAMMA = 0.9 #reward-discount
EPS = 0.1 #epsilon-greedy
TARGET_UPDATE = 20 #policy to target

optimizer = optim.SGD(policy_net.parameters(), lr=0.1)
loss_fn = torch.nn.MSELoss()

## 5. Optimize



In [129]:
def optimize_model():
    if len(experience_pool) < BATCH_SIZE:
        return #not enough experience

    experiences = experience_pool.sample(BATCH_SIZE)
    experiences_batch = Experience(*zip(*experiences)) #experiences of batches

    state_batch = torch.stack(experiences_batch.state)
    action_batch = torch.stack(experiences_batch.action)
    reward_batch = torch.stack(experiences_batch.reward)
    next_state_batch = torch.stack(experiences_batch.next_state)
    terminal_batch = torch.stack(experiences_batch.terminal)

    output_policy = policy_net(state_batch)
    policy_q_value = torch.gather(output_policy, 1, action_batch)

    with torch.no_grad():
        output_target_next = target_net(next_state_batch)
        target_next_q_value = torch.max(output_target_next, dim=1).values

    target_q_value = reward_batch + GAMMA * target_next_q_value * (1 - terminal_batch)

    # mean square loss
    loss = loss_fn(policy_q_value, target_q_value )

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## 6. Exploration

In [127]:
def greedy_action(state): # state is tensor
    with torch.no_grad():
        action = torch.argmax(policy_net(state)).item()
    return action #return integer

def sample_action(state):
    if np.random.uniform(0, 1) < (1.0 - EPS): #choose a by policy-NN
        action = greedy_action(state) #greedy
    else:
        action = env.action_space.sample() #random action

    return torch.tensor([action], dtype=torch.int) #return tensor

def gen_experience(state, pool):
    action = sample_action(state) # a
    obs, reward, done, _ = env.step(action.item())
    reward = torch.tensor(reward) # r
    next_state = torch.tensor(obs) # s'
    terminal = torch.tensor(int(done), dtype=torch.int) # t

    # Store the transition in experience pool
    pool.push(state, action, reward, next_state, terminal) #(s,a,r,s',t), tensors

    return done, next_state


experience_pool.push(torch.tensor([1.0,2.0,3.0,4.0]), torch.tensor([1]), torch.tensor(1.0), torch.tensor([5.0,6.0,7.0,8.0]), torch.tensor(0))
experience_pool.push(torch.tensor([5.0,6.0,7.0,8.0]), torch.tensor([0]), torch.tensor(0.0), torch.tensor([1.0,2.0,3.0,4.0]), torch.tensor(11))
print(len(experience_pool))




6
tensor([[0.3633, 0.7404],
        [0.0888, 0.3651]], grad_fn=<AddmmBackward>)
tensor([[0],
        [1]])


tensor([[0.3633],
        [0.3651]], grad_fn=<GatherBackward>)

## 7. Train Loop

In [ ]:
EPOCHS = 100
for epoch in range(EPOCHS):
    # Initialize the environment and state
    state = torch.tensor(env.reset()) # s

    while True:
        # generate experience
        done, next_state = gen_experience(state, experience_pool)
        state = next_state
        # Perform one step of the optimization
        optimize_model()
        # one episode
        if done:
            break

    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
